## 1.Datenaufbereitung (Preprocessing)

In [1]:
import pandas as pd

# Liste der Dateipfade mit dem relativen Pfad '../'
file_paths = [
    '../data/audi.csv',
    '../data/bmw.csv',
    '../data/cclass.csv',
    '../data/focus.csv',
    '../data/ford.csv',
    '../data/hyundi.csv',
    '../data/merc.csv',
    '../data/skoda.csv',
    '../data/toyota.csv',
    '../data/vauxhall.csv',
    '../data/vw.csv',
]

# Liste, um alle DataFrames zu speichern
all_dataframes = []

# Durchlaufe alle Dateipfade und lade die CSV-Dateien
for file_path in file_paths:
    df = pd.read_csv(file_path)
    brand = file_path.split('/')[-1].split('.')[0]  # Extrahiere die Marke aus dem Dateinamen
    df['brand'] = brand  # Füge eine Spalte mit der Marke hinzu
    # Ändere die Reihenfolge der Spalten, um "brand" an den Anfang zu setzen
    df = df[['brand'] + [col for col in df.columns if col != 'brand']]
    all_dataframes.append(df)

# Verbinde alle DataFrames
combined_df = pd.concat(all_dataframes, ignore_index=True)

# Speichere den kombinierten DataFrame im Ordner "data (clean)"
# combined_df.to_csv('../data (clean)/combined.csv', index=False)

In [2]:
combined_df

,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,audi,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,NaN
1,audi,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,NaN
2,audi,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,NaN
3,audi,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,NaN
4,audi,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
108535,vw,Eos,2012,5990,Manual,74000,Diesel,125.0,58.9,2.0,NaN
108536,vw,Fox,2008,1799,Manual,88102,Petrol,145.0,46.3,1.2,NaN
108537,vw,Fox,2009,1590,Manual,70000,Petrol,200.0,42.0,1.4,NaN
108538,vw,Fox,2006,1250,Manual,82704,Petrol,150.0,46.3,1.2,NaN
